In [1]:
!pip install --no-deps -qq /kaggle/input/dicomsdl--0-109-2/dicomsdl-0.109.2-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
!pip install --no-deps -qq /kaggle/input/segmentation-models-pytorch-021/wheels/munch-2.5.0-py2.py3-none-any.whl
!pip install --no-deps -qq /kaggle/input/segmentation-models-pytorch-021/wheels/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install --no-deps -qq /kaggle/input/segmentation-models-pytorch-021/wheels/timm-0.4.12-py3-none-any.whl
!pip install --no-deps -qq /kaggle/input/segmentation-models-pytorch-021/wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl
!pip install --no-deps -qq /kaggle/input/segmentation-models-pytorch-021/wheels/segmentation_models_pytorch-0.2.1-py3-none-any.whl

In [2]:
import os
import cv2
import glob
import gc
import sys
import pydicom
import dicomsdl
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import gzip

from tqdm import tqdm
from joblib import Parallel, delayed
from pydicom.pixel_data_handlers.util import apply_voi_lut
from multiprocessing import Pool

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.cuda.amp as amp
import segmentation_models_pytorch as smp

sys.path.append('/kaggle/input/resnet-3d-rsna-atd')
sys.path.append('/kaggle/input/covn3d-same')
sys.path.append('/kaggle/input/rsna-atd-lib')
from resnet3d import generate_model
import timm
import timm_new

#For dataloader using cuda
torch.multiprocessing.set_start_method('spawn')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Parameters

In [3]:
RESOL = 160
BATCH_SIZE = 4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEBUG = False
N_DEBUG_SAMPLES = 20

UP_RESOL = 128
N_CHANNELS = 6
PREPROC_NORM_OR_STD = False # True: normalization, False: standardization

BASE_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
SAVE_PATH = '/tmp'
MASK_SAVE_PATH = f'{SAVE_PATH}/mask_preprocessed'
MASK_VALID_PATH = f'{SAVE_PATH}/mask_validation'

DATA_PATH = f'{BASE_PATH}/test_images'
if DEBUG:
    DATA_PATH = f'{BASE_PATH}/train_images'
N_PREPROCESS_CHUNKS = 4
N_PROCESS_CROP = 4

#kernel_type = 'timm3d_res50d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep'
#kernel_type = 'timm3d_res50d_unet4b_128_128_128_dsv2_flip12_shift333p7_gd1p5_bs4_lr3e4_20x50ep_fold0_best_0.9.pth'
seg_model_path = '/kaggle/input/rsna-atd-weights/231001_timm3d_res10tc_CV0.938.pt'
load_kernel = None
load_last = True
n_blocks = 4
backbone = 'timm/resnet10t.c3_in1k'

backbone_classifier = 'timm/resnetrs50.tf_in1k'
# weights_classifier = '/kaggle/input/1007-resnetrs50-cv0460/timm_resnetrs50.tf_in1k_lr0.0002_epochs_500_resol128_batch12-cv0460.pt'
weights_classifier = '/kaggle/input/1007-resnetrs50-cv0460/timm_resnetrs50.tf_in1k_lr0.0002_epochs_500_resol128_batch12-cv0454.pt'
weights_classifier1_2 = '/kaggle/input/1010-resnet50-cv04827/timm_resnetrs50.tf_in1k_lr0.0002_epochs_500_resol128_batch12_fold1-cv04827.pt'

backbone_classifier2 = 'timm/resnet10t.c3_in1k'
weights_classifier2 = '/kaggle/input/rsna-atd-weights/231002_timm_resnet10t.c3_in1k_lr0.005_epochs_200_resol128_batch24.pt2'

backbone_classifiers3 = 'timm/resnet10t.c3_in1k'
weights_classifiers3 = '/kaggle/input/rsna-atd-weights/timm_resnet10t_LSTM_CV0.44.pt'


data_dir = '../input/rsna-2022-cervical-spine-fracture-detection'
use_amp = True
num_workers = 0
out_dim = 5
n_blocks = 4
drop_rate = 0.2
drop_path_rate = 0.2
p_mixup = 0.0

chan_keys = ['bowel', 'left_kidney', 'right_kidney', 'liver', 'spleen', 'total']
chan_dict = {}
for i in range(0, 6):
    chan_dict[i] = chan_keys[i]

log_dir = f'{SAVE_PATH}/seg_models_backup'
model_dir = f'{SAVE_PATH}/seg_models_backup'
seg_inference_dir = f'{SAVE_PATH}/seg_infer_results'
origin_img_dir = f'{SAVE_PATH}/3d_preprocessed'
cropped_img_dir   = f'{SAVE_PATH}/3d_preprocessed_crop'
os.makedirs(log_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(MASK_VALID_PATH, exist_ok=True)
os.makedirs(seg_inference_dir, exist_ok = True)
os.makedirs(cropped_img_dir, exist_ok = True)
os.makedirs(origin_img_dir, exist_ok = True)
DEVICE

device(type='cuda')

In [4]:
transforms_valid = transforms.Compose([
])

In [5]:
train_df = pd.read_csv(f'{BASE_PATH}/train.csv')
train_meta = pd.read_csv(f'{BASE_PATH}/train_series_meta.csv')
train_df = train_df.sort_values(by=['patient_id'])

n_chunk = 8
patients = os.listdir(DATA_PATH)
n_patients = len(patients)
rng_patients = np.linspace(0, n_patients+1, n_chunk+1, dtype = int)
patients_cts = glob.glob(f'{DATA_PATH}/*/*')
n_cts = len(patients_cts)
patients_cts_arr = np.zeros((n_cts, 2), int)
data_paths=[]
for i in range(0, n_cts):
    patient, ct = patients_cts[i].split('/')[-2:]
    patients_cts_arr[i] = patient, ct
    data_paths.append(f'{SAVE_PATH}/3d_preprocessed/{patients_cts_arr[i,0]}_{patients_cts_arr[i,1]}.pkl')
TRAIN_IMG_PATH = BASE_PATH + '/processed' 

#Generate tables for training
df_data = pd.DataFrame(patients_cts_arr, columns = ['patient_id', 'series'])

#5-fold splitting
train_df['fold'] = 0
labels = train_df[['bowel_healthy','bowel_injury',
                    'extravasation_healthy','extravasation_injury',
                    'kidney_healthy','kidney_low','kidney_high',
                    'liver_healthy','liver_low','liver_high',
                    'spleen_healthy','spleen_low','spleen_high',
                    'any_injury']].to_numpy()


#df_data = df_data.join(train_df.set_index('patient_id'), on='patient_id')
df_data['path']=data_paths

#For mask paths
mask_paths = []
cropped_paths = []
for i in range(0, len(df_data)):
    row = df_data.iloc[i]
    file_name = row['path'].split('/')[-1]
    mask_paths.append(f'{seg_inference_dir}/{file_name}')
    cropped_paths.append(f'{cropped_img_dir}/{file_name}')
df_data['mask_path'] = mask_paths
df_data['cropped_path'] = cropped_paths

df_data.to_csv(f'{SAVE_PATH}/data_meta.csv', index = False)


In [6]:
df_data

,patient_id,series,path,mask_path,cropped_path
0,63706,39279,/tmp/3d_preprocessed/63706_39279.pkl,/tmp/seg_infer_results/63706_39279.pkl,/tmp/3d_preprocessed_crop/63706_39279.pkl
1,50046,24574,/tmp/3d_preprocessed/50046_24574.pkl,/tmp/seg_infer_results/50046_24574.pkl,/tmp/3d_preprocessed_crop/50046_24574.pkl
2,48843,62825,/tmp/3d_preprocessed/48843_62825.pkl,/tmp/seg_infer_results/48843_62825.pkl,/tmp/3d_preprocessed_crop/48843_62825.pkl


# Preprocess images to 3D data

## Parameters
Reference: https://www.kaggle.com/code/theoviel/get-started-quicker-dicom-png-conversion

In [7]:
BASE_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
TEST_PATH = f'{BASE_PATH}/train_images/'

print('Number of test patients :', len(os.listdir(TEST_PATH)))

Number of test patients : 3147


In [8]:
n_chunk = 4
patients = os.listdir(TEST_PATH)
#patients = patients[:8]
n_patients = len(patients)

#if(n_patients==3):
#    n_chunk = 3
#    rng_patients = np.linspace(0, n_patients, n_chunk+1, dtype = int)
#else:
rng_patients = np.linspace(0, n_patients, n_chunk+1, dtype = int)

#rng_patients
#patients
#if DEBUG:
#    rng_patients = [0, n_patients]

In [9]:
def compress(name, data):
    with gzip.open(name, 'wb') as f:
        pickle.dump(data, f)

def decompress(name):
    with gzip.open(name, 'rb') as f:
        data = pickle.load(f)
    return data

def compress_fast(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

def decompress_fast(name):
    with open(name, 'rb') as f:
        data = pickle.load(f)
    return data

In [10]:
data_path = TEST_PATH
save_folder = '/tmp/3d_processed'

try:
    os.makedirs(save_folder)
except:
    print('save folder already exists!')

# Data & Model

In [11]:
#Returns GPU array
def standardize_pixel_array(pixel_array, dcm_rows):
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    for z in range(0, len(pixel_array)):
        if int(dcm_rows[z]['PixelRepresentation']) == 1:
            bit_shift = dcm_rows[z]['BitsAllocated'] - dcm_rows[z]['BitsStored']
            dtype = pixel_array[z].dtype 
            pixel_array[z] = (pixel_array[z] << bit_shift).astype(dtype) >>  bit_shift

    pixel_array = torch.from_numpy(pixel_array.astype(np.float16)).to(DEVICE).to(torch.float16)    

    for z in range(0, len(pixel_array)):
        intercept = float(dcm_rows[z]['RescaleIntercept'])
        slope = float(dcm_rows[z]['RescaleSlope'])
        center = int(dcm_rows[z]['WindowCenter'])
        width = int(dcm_rows[z]['WindowWidth'])
        low = center - width / 2
        high = center + width / 2    
        
        pixel_array[z] = (pixel_array[z] * slope) + intercept
        pixel_array[z] = torch.clip(pixel_array[z], low, high)
        
    gc.collect()    
    return pixel_array

In [12]:
def resize_norm_or_std(data, resize_shape, is_norm = PREPROC_NORM_OR_STD):  
    #resize xy
    data = transforms.Resize((int(resize_shape[1]), int(resize_shape[2])), antialias = True)(data)
    
    #zyx to xzy
    data = torch.permute(data, (2, 0, 1))
    #Resize yz
    data = transforms.Resize((int(resize_shape[0]), int(resize_shape[1])), antialias = True)(data)
    #xzy to zyx
    data = torch.permute(data, (1, 2, 0))

    if is_norm:
        bottom = torch.min(data)
        data -= bottom
        top    = torch.max(data)
        data/=top
        del top, bottom
    else:
        avg = torch.mean(data, (0, 1, 2))
        std = torch.std(data, (0, 1, 2))
        data = (data-avg)/std
        del avg, std

    gc.collect()
    #torch.cuda.empty_cache()
    return data

# Read each slice and stack them to make 3d data
def process_3d(save_path, data_path = DATA_PATH):
    tmp = save_path.split('/')[-1][:-4]
    tmp = tmp.split('_')
    patient, study = int(tmp[0]), int(tmp[1])
    imgs = {}    
    
    # To load only needed slices
    imgs = {}    
    for f in sorted(glob.glob(data_path + f'/{patient}/{study}/*.dcm')):  
        pos_z = -int((f.split('/')[-1])[:-4])
        imgs[pos_z] = f
        
    sample_z = np.linspace(0, len(imgs)-1, RESOL, dtype=int)
    dcm_rows = []
    imgs_3d  = []
    for i, k in enumerate(sorted(imgs.keys())):
        if not np.isin([i], sample_z)[0]:
            continue        
        f= imgs[k]
        opened_dicom = dicomsdl.open(f)
        img = opened_dicom.pixelData(storedvalue=True)
        params = opened_dicom.getPixelDataInfo()
        
        imgs_3d.append(img[None])
        dcm_rows.append(params)

    imgs_3d = np.vstack(imgs_3d)
    imgs_3d = standardize_pixel_array(imgs_3d, dcm_rows)
    
    min_imgs = torch.min(imgs_3d)
    max_imgs = torch.max(imgs_3d)
        
    imgs_3d = ((imgs_3d - min_imgs) / (max_imgs - min_imgs + 1e-6))

    if str(dcm_rows[0]['PhotometricInterpretation']) == "MONOCHROME1":
        imgs_3d = 1.0 - imgs_3d

    imgs_3d = resize_norm_or_std(imgs_3d, [RESOL, RESOL, RESOL])

    #Save the image
    #compress(save_path, imgs_3d)         4             

    del imgs, img
    gc.collect()
    return imgs_3d

In [13]:
class SEGDataset(Dataset):
    def __init__(self, df, mode, transform):

        self.df = df.reset_index()
        self.mode = mode
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        #try:
        #    data_3d = decompress(row['path'])
        #except:
        data_3d = process_3d(row['path']).unsqueeze(0).to(torch.float16)
            
        #data_3d = torch.from_numpy(data_3d).to(torch.float32)
        #file_name = row['path'].split('/')[-1]
        #save_path = f'{seg_inference_dir}/{file_name}'
        save_path = row['mask_path']

        return data_3d, save_path

# Segmentation models

In [14]:
class TimmSegModel(nn.Module):
    def __init__(self, backbone, segtype='unet', pretrained=False):
        super(TimmSegModel, self).__init__()

        self.encoder = timm_new.create_model(
            backbone,
            in_chans=1,
            features_only=True,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            pretrained=False
        )
        g = self.encoder(torch.rand(1, 1, 64, 64))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]
        if segtype == 'unet':
            self.decoder = smp.unet.decoder.UnetDecoder(
                encoder_channels=encoder_channels[:n_blocks+1],
                decoder_channels=decoder_channels[:n_blocks],
                n_blocks=n_blocks,
            )

        self.segmentation_head = nn.Conv2d(decoder_channels[n_blocks-1], out_dim, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

    def forward(self,x):
        global_features = [0] + self.encoder(x)[:n_blocks]
        seg_features = self.decoder(*global_features)
        seg_features = self.segmentation_head(seg_features)
        return seg_features

In [15]:
from timm.models.layers.conv2d_same import Conv2dSame
from conv3d_same import Conv3dSame


def convert_3d(module):

    module_output = module
    if isinstance(module, torch.nn.BatchNorm2d):
        module_output = torch.nn.BatchNorm3d(
            module.num_features,
            module.eps,
            module.momentum,
            module.affine,
            module.track_running_stats,
        )
        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig
            
    elif isinstance(module, Conv2dSame):
        module_output = Conv3dSame(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.Conv2d):
        module_output = torch.nn.Conv3d(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
            padding_mode=module.padding_mode
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.MaxPool2d):
        module_output = torch.nn.MaxPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            ceil_mode=module.ceil_mode,
        )
    elif isinstance(module, torch.nn.AvgPool2d):
        module_output = torch.nn.AvgPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            ceil_mode=module.ceil_mode,
        )

    for name, child in module.named_children():
        module_output.add_module(
            name, convert_3d(child)
        )
    del module

    return module_output

# Semgmentation inference

In [16]:
def infer_func(model, loader_valid):
    model.eval()
    ths = [0.1]
    bar = tqdm(loader_valid)
    counter = 0
    
    with torch.no_grad():
        with amp.autocast():
            for images, save_paths in bar:
                images = images.cuda()
                logits = model(images)
                for thi, th in enumerate(ths):
                    for i in range(logits.shape[0]):                    
                        y_pred = ((logits[i].sigmoid()> th).float().detach().cpu().numpy()+0.1).astype(np.uint8)
                        compress(save_paths[i], y_pred)
    del images, logits, y_pred
    gc.collect()
    torch.cuda.empty_cache()

In [17]:
df_data = pd.read_csv(f'{SAVE_PATH}/data_meta.csv')
if DEBUG:
    df_data = df_data.iloc[:N_DEBUG_SAMPLES]

mask_paths = []
cropped_paths = []
for i in range(0, len(df_data)):
    row = df_data.iloc[i]
    file_name = row['path'].split('/')[-1]
    mask_paths.append(f'{seg_inference_dir}/{file_name}')
    cropped_paths.append(f'{cropped_img_dir}/{file_name}')
df_data['mask_path'] = mask_paths
df_data['cropped_path'] = cropped_paths
df_data.tail()

df_data.to_csv(f'{SAVE_PATH}/data_meta.csv', index = False)

In [18]:
def run(fold):
    model_file = seg_model_path

    dataset_train = SEGDataset(df_data, 'valid', transform=transforms_valid)
    loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

    model = TimmSegModel(backbone, pretrained=True)
    model = convert_3d(model)

    model.load_state_dict(torch.load(model_file))
    model = model.to(DEVICE)

    print(len(dataset_train))

    infer_func(model, loader_train)

    del model, dataset_train, loader_train
    gc.collect()
    torch.cuda.empty_cache()

In [19]:
run(0)

3


100%|██████████| 1/1 [00:10<00:00, 10.33s/it]


In [20]:
#!cp /tmp/seg_infer_results/26883_57967* /kaggle/working

## New Crop operations

In [21]:
#Returns GPU array
def standardize_pixel_array(pixel_array, dcm_rows):
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    #pixel_array = dcm.pixel_array
    #pixel_array = cp.array(pixel_array)    
    for z in range(0, len(pixel_array)):
        if int(dcm_rows[z]['PixelRepresentation']) == 1:
            bit_shift = dcm_rows[z]['BitsAllocated'] - dcm_rows[z]['BitsStored']
            dtype = pixel_array[z].dtype 
            pixel_array[z] = (pixel_array[z] << bit_shift).astype(dtype) >>  bit_shift
    #         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

    pixel_array = torch.from_numpy(pixel_array.astype(np.float16)).to(DEVICE).to(torch.float32)    

    for z in range(0, len(pixel_array)):
        intercept = float(dcm_rows[z]['RescaleIntercept'])
        slope = float(dcm_rows[z]['RescaleSlope'])
        center = int(dcm_rows[z]['WindowCenter'])
        width = int(dcm_rows[z]['WindowWidth'])
        low = center - width / 2
        high = center + width / 2    
        
        pixel_array[z] = (pixel_array[z] * slope) + intercept
        pixel_array[z] = torch.clip(pixel_array[z], low, high)
        
    gc.collect()
    
    return pixel_array


In [22]:
#The order of the crop region data format
#Z start/end, Y start/end, X start/end for each mask channels + total region for the extravasation prediction
def calc_crop_region(mask):
    crop_range = np.zeros((6, 6))
    crop_range[:,::2]=10000
    mask_z = np.max(mask, axis = (2, 3)).astype(bool)
    mask_y = np.max(mask, axis = (1, 3)).astype(bool)
    mask_x = np.max(mask, axis = (1, 2)).astype(bool)
    
    template_range = np.arange(0, RESOL)

    for mi in range(0, 5):
        zrange = template_range[mask_z[mi]]
        yrange = template_range[mask_y[mi]]
        xrange = template_range[mask_x[mi]]
        # For incomplete organ
        if(len(zrange)==0):
            zrange = template_range.copy()
            yrange = template_range.copy()
            xrange = template_range.copy()

        crop_range[mi] = np.min(zrange), np.max(zrange)+1, np.min(yrange), np.max(yrange)+1, np.min(xrange), np.max(xrange)+1

    crop_range[5] = np.min(crop_range[:5, 0]), np.max(crop_range[:5, 1]), np.min(crop_range[:5, 2]), \
                    np.max(crop_range[:5, 3]), np.min(crop_range[:5,4]), np.max(crop_range[:5, 5])
    
    crop_range[:,:2]/=len(mask_z[0])
    crop_range[:,2:4]/=len(mask_y[0])
    crop_range[:,4:6]/=len(mask_x[0])

    # Then make extravasation (# 5 mask) to reference one and convert other mask's crop respective to it
    # --> To minimize the loading size due to speed issue.
    zmin, rel_zrange = crop_range[5,0], crop_range[5,1]-crop_range[5,0]
    ymin, rel_yrange = crop_range[5,2], crop_range[5,3]-crop_range[5,2]
    xmin, rel_xrange = crop_range[5,4], crop_range[5,5]-crop_range[5,4]

    crop_range[:5,:2] = (crop_range[:5,:2]-zmin)/rel_zrange
    crop_range[:5,2:4] = (crop_range[:5,2:4]-ymin)/rel_yrange
    crop_range[:5,4:6] = (crop_range[:5,4:6]-xmin)/rel_xrange

    return crop_range

def crop_resize_avg_and_std_3d(data, region, resize_shape):  
    shapes = data.shape
    region[:2]*=shapes[0]
    region[2:4]*=shapes[1]
    region[4:6]*=shapes[2]
    region = region.astype(int)

    cropped = torch.clone(data[region[0]:region[1], region[2]:region[3], region[4]:region[5]])    

    #resize xy
    cropped = transforms.Resize((int(resize_shape[1]), int(resize_shape[2])), antialias = True)(cropped)
    #slices = []
    #for i in range(0, len(cropped)):
    #    slices.append(cv2.resize(cropped[i], (resize_shape[2], resize_shape[1]))[None])
    
    #slices = np.vstack(slices)
    #resized_cropped = np.zeros(resize_shape)
    
    #zyx to xzy
    cropped = torch.permute(cropped, (2, 0, 1))
    cropped = transforms.Resize((int(resize_shape[0]), int(resize_shape[1])), antialias = True)(cropped)
    #xzy to zyx
    cropped = torch.permute(cropped, (1, 2, 0))
    #for i in range(0, len(slices[0,0])):
    #    resized_cropped[:,:,i] = cv2.resize(slices[:,:,i], (resize_shape[1], resize_shape[0]))

        
    #std = torch.std(cropped, (0, 1, 2))
    #avg = torch.mean(cropped, (0, 1, 2))
    min_imgs = torch.min(cropped)
    max_imgs = torch.max(cropped)
    #for debugging   
    cropped = ((cropped - min_imgs) / (max_imgs - min_imgs + 1e-6))
    
    del min_imgs, max_imgs, shapes, region
    gc.collect()
    torch.cuda.empty_cache()
    return cropped

# Read each slice and stack them to make 3d data
def process_3d_crop(save_path, mask_path, resize_shapes, data_path = DATA_PATH):
    tmp = save_path.split('/')[-1][:-4]
    tmp = tmp.split('_')
    patient, study = int(tmp[0]), int(tmp[1])
    
    mask = decompress(mask_path)
    crop_regions = calc_crop_region(mask)
    absolute_crop = crop_regions[5].copy() # To load minimum pixels...

    del mask
    gc.collect()
    crop_regions[5] = 0, 1, 0, 1, 0, 1

    imgs = {}    
    
    for f in sorted(glob.glob(data_path + f'/{patient}/{study}/*.dcm')):  
        pos_z = -int((f.split('/')[-1])[:-4])
        imgs[pos_z] = f

    imgs_3d = []
    n_imgs = len(imgs)    
    z_crop_range= (absolute_crop[0:2]*n_imgs).astype(int)
    #print(z_crop_range)
    
    dcm_rows = []
    for i, k in enumerate(sorted(imgs.keys())):
        #if i in sample_z:
        if(i >= z_crop_range[0] and i < z_crop_range[1]):
            IS_XY_CROP = False
            f = imgs[k]
            #Exception for the corrupted dicom file
            if (f=='/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images/3124/5842/514.dcm'):
                continue
            #try:            
            opened_dicom = dicomsdl.open(f)
            img = opened_dicom.pixelData(storedvalue=True)
            params = opened_dicom.getPixelDataInfo()

            if not IS_XY_CROP:
                img_shape = np.shape(img)
                xy_crop_range = absolute_crop[2:].copy()   
                xy_crop_range[0:2]*=img_shape[0]
                xy_crop_range[2:4]*=img_shape[1]            
                xy_crop_range = xy_crop_range.astype(int)                
                IS_XY_CROP = True
                
            img = img[xy_crop_range[0]:xy_crop_range[1], xy_crop_range[2]:xy_crop_range[3]]             

            #dcm_row = pd.DataFrame.from_dict(params)                   
            dcm_rows.append(params)                  
            imgs_3d.append(img[None])

    del opened_dicom
    gc.collect()
                
    imgs_3d = np.vstack(imgs_3d)

    imgs_3d  = standardize_pixel_array(imgs_3d, dcm_rows)

    min_imgs = torch.min(imgs_3d)
    max_imgs = torch.max(imgs_3d)
    for i in range(0, 2):
        min_imgs = torch.min(min_imgs)
        max_imgs = torch.max(max_imgs)
        
    imgs_3d = ((imgs_3d - min_imgs) / (max_imgs - min_imgs + 1e-6))

    #print(dcm_rows[0].PhotometricInterpretation)
    if str(dcm_rows[0]['PhotometricInterpretation']) == "MONOCHROME1":
        imgs_3d = 1.0 - imgs_3d

    #Loaded original imgs_3d    
    #processed_img_3d = np.zeros((6, RESOL, RESOL, RESOL))
    
    origin_shape = imgs_3d.shape
    for i in range(0, 6):    
        #To deal with almost not detected slices
        try:   
            # To deal with possible noises
            if(((crop_regions[i,1]-crop_regions[i,0]) < 10/origin_shape[0]) or 
                ((crop_regions[i,3]-crop_regions[i,2]) < 10/origin_shape[1]) or 
                ((crop_regions[i,5]-crop_regions[i,4]) < 10/origin_shape[2])):
                dummy_failure_function()
            
            processed_img_3d = (crop_resize_avg_and_std_3d(imgs_3d, crop_regions[i], resize_shapes[i])).to(torch.float16).to('cpu')
            compress_fast(f'{save_path}_{i}', processed_img_3d)      

            del processed_img_3d
            gc.collect()
        except:
            processed_img_3d = (crop_resize_avg_and_std_3d(imgs_3d, np.array([0, 1, 0, 1, 0, 1]), resize_shapes[i])).to(torch.float16).to('cpu')
            compress_fast(f'{save_path}_{i}', processed_img_3d)
            del processed_img_3d
            gc.collect()  

    del imgs, img, imgs_3d
    gc.collect()
    torch.cuda.empty_cache()

In [23]:
resize_shapes = np.zeros((6, 3), int)
resize_shapes[0] = 131, 107, 148
resize_shapes[1] = 107, 129, 150
resize_shapes[2] = 105, 138, 144
resize_shapes[3] = 77, 150, 180
resize_shapes[4] =  85, 155, 158
resize_shapes[5] = 123, 109, 155

# Preprocess dataset
rng_samples = np.linspace(0, len(df_data), N_PROCESS_CROP+1, dtype = int)
def process_3d_wrapper(process_ind, rng_samples = rng_samples, data_meta_df = df_data, resize_shapes = resize_shapes):
    for i in tqdm(range(rng_samples[process_ind], rng_samples[process_ind+1])):
        process_3d_crop(data_meta_df.iloc[i]['cropped_path'], data_meta_df.iloc[i]['mask_path'], resize_shapes)

In [24]:
%%time
Parallel(n_jobs = N_PROCESS_CROP)(delayed(process_3d_wrapper)(i) for i in range(N_PROCESS_CROP))

0it [00:00, ?it/s]
100%|██████████| 1/1 [00:06<00:00,  6.95s/it]


CPU times: user 208 ms, sys: 289 ms, total: 498 ms
Wall time: 16.3 s


100%|██████████| 1/1 [00:07<00:00,  7.50s/it]


[None, None, None, None]

# Final inference model

In [25]:
def convert_3d(module):
    module_output = module
    if isinstance(module, torch.nn.BatchNorm2d):
        module_output = torch.nn.BatchNorm3d(
            module.num_features,
            module.eps,
            module.momentum,
            module.affine,
            module.track_running_stats,
        )
        if module.affine:
            with torch.no_grad():
                module_output.weight = module.weight
                module_output.bias = module.bias
        module_output.running_mean = module.running_mean
        module_output.running_var = module.running_var
        module_output.num_batches_tracked = module.num_batches_tracked
        if hasattr(module, "qconfig"):
            module_output.qconfig = module.qconfig
            
    elif isinstance(module, Conv2dSame):
        module_output = Conv3dSame(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.Conv2d):
        module_output = torch.nn.Conv3d(
            in_channels=module.in_channels,
            out_channels=module.out_channels,
            kernel_size=module.kernel_size[0],
            stride=module.stride[0],
            padding=module.padding[0],
            dilation=module.dilation[0],
            groups=module.groups,
            bias=module.bias is not None,
            padding_mode=module.padding_mode
        )
        module_output.weight = torch.nn.Parameter(module.weight.unsqueeze(-1).repeat(1,1,1,1,module.kernel_size[0]))

    elif isinstance(module, torch.nn.MaxPool2d):
        module_output = torch.nn.MaxPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            dilation=module.dilation,
            ceil_mode=module.ceil_mode,
        )
    elif isinstance(module, torch.nn.AvgPool2d):
        module_output = torch.nn.AvgPool3d(
            kernel_size=module.kernel_size,
            stride=module.stride,
            padding=module.padding,
            ceil_mode=module.ceil_mode,
        )

    for name, child in module.named_children():
        module_output.add_module(
            name, convert_3d(child)
        )
    del module

    return module_output


class Timm3DModel(nn.Module):
    def __init__(self, backbone, n_channels, n_labels, segtype='unet', pretrained=False):
        super(Timm3DModel, self).__init__()
        self.n_labels = n_labels
        self.encoder = timm_new.create_model(
            backbone,
            in_chans=n_channels,
            features_only=True,
            drop_rate=drop_rate,
            drop_path_rate=drop_path_rate,
            pretrained=pretrained
        )
        g = self.encoder(torch.rand(1, n_channels, 64, 64))
        encoder_channels = [1] + [_.shape[1] for _ in g]
        decoder_channels = [256, 128, 64, 32, 16]

        self.avgpool = nn.AvgPool2d(5, 4, 2)
        
        [_.shape[1] for _ in g]
        self.convs1x1 = nn.ModuleList()    
        self.batchnorms = nn.ModuleList()    
        self.batchnorms13 = nn.ModuleList()
        for i in range(0, len(g)):
            self.convs1x1.append(nn.Conv2d(g[i].shape[1], self.n_labels, 1))
        del g
        gc.collect()
        
    def forward(self,x):
        batch_size = x.shape[0]
        global_features = self.encoder(x)[:n_blocks]        
        for i in range(0, len(global_features)):
            global_features[i] = self.convs1x1[i](global_features[i])
        return global_features
    
    
class Timm3DModelClassifier(nn.Module):
    def __init__(self, backbone, n_channels, n_labels, segtype='unet', pretrained=False):
        super(Timm3DModelClassifier, self).__init__()
        self.model_3d = Timm3DModel(backbone, n_channels, n_labels, segtype, pretrained)
        self.model_3d = convert_3d(self.model_3d)
        self.n_channels = n_channels
        self.n_labels = n_labels    
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.model_3d(x)
        pooled_features = []
        for i in range(0, len(x)):
            pooled_features.append(torch.reshape(torch.mean(x[i], dim = (2, 3, 4)), (batch_size, self.n_labels, 1)))
        pooled_features = torch.cat(pooled_features, dim=2)     
        labels = torch.mean(pooled_features, dim = 2)
        return labels
    
class AbdominalClassifier(nn.Module):
    def __init__(self, device = DEVICE):
        super().__init__()
        self.device = device
        self.upsample = torch.nn.Upsample(size = [UP_RESOL, UP_RESOL, UP_RESOL])
        
        self.model3d_bowel        = Timm3DModelClassifier(backbone_classifier, 1, 2)      
        self.model3d_extrav       = Timm3DModelClassifier(backbone_classifier, 1, 2)
        self.model3d_kidney_left  = Timm3DModelClassifier(backbone_classifier, 1, 3)
        self.model3d_kidney_right = Timm3DModelClassifier(backbone_classifier, 1, 3)
        self.model3d_liver        = Timm3DModelClassifier(backbone_classifier, 1, 3)
        self.model3d_spleen       = Timm3DModelClassifier(backbone_classifier, 1, 3)
        
        self.flatten  = nn.Flatten()
        self.dropout  = nn.Dropout(p=0.5)
        self.softmax  = nn.Softmax(dim=1)        
        self.maxpool  = nn.MaxPool1d(5, 1)
        
    def forward(self, x_bowel, x_kidney_left, x_kidney_right, x_liver, x_spleen, x_total):
        bowel_label        = self.model3d_bowel(x_bowel)
        extrav_label       = self.model3d_extrav(x_total)
        kidney_label_left  = self.model3d_kidney_left(x_kidney_left)
        kidney_label_right = self.model3d_kidney_right(x_kidney_right)
        kidney_label       = (kidney_label_left + kidney_label_right)/2
        liver_label        = self.model3d_liver(x_liver)
        spleen_label       = self.model3d_spleen(x_spleen)
        
        bowel_soft = self.softmax(bowel_label)
        extrav_soft = self.softmax(extrav_label)
        kidney_soft = self.softmax(kidney_label)
        liver_soft = self.softmax(liver_label)
        spleen_soft = self.softmax(spleen_label)
        
        labels = torch.cat([bowel_soft, extrav_soft, kidney_soft, liver_soft, spleen_soft], dim = 1)

        return labels
    
    
class AbdominalClassifier2(nn.Module):
    def __init__(self, device = DEVICE):
        super().__init__()
        self.device = device
        self.upsample = torch.nn.Upsample(size = [UP_RESOL, UP_RESOL, UP_RESOL])
        
        self.model3d_bowel        = Timm3DModelClassifier(backbone_classifier2, 1, 2)      
        self.model3d_extrav       = Timm3DModelClassifier(backbone_classifier2, 1, 2)
        self.model3d_kidney_left  = Timm3DModelClassifier(backbone_classifier2, 1, 3)
        self.model3d_kidney_right = Timm3DModelClassifier(backbone_classifier2, 1, 3)
        self.model3d_liver        = Timm3DModelClassifier(backbone_classifier2, 1, 3)
        self.model3d_spleen       = Timm3DModelClassifier(backbone_classifier2, 1, 3)
        
        self.flatten  = nn.Flatten()
        self.dropout  = nn.Dropout(p=0.5)
        self.softmax  = nn.Softmax(dim=1)        
        self.maxpool  = nn.MaxPool1d(5, 1)
        
    def forward(self, x_bowel, x_kidney_left, x_kidney_right, x_liver, x_spleen, x_total):
        bowel_label        = self.model3d_bowel(x_bowel)
        extrav_label       = self.model3d_extrav(x_total)
        kidney_label_left  = self.model3d_kidney_left(x_kidney_left)
        kidney_label_right = self.model3d_kidney_right(x_kidney_right)
        kidney_label       = (kidney_label_left + kidney_label_right)/2
        liver_label        = self.model3d_liver(x_liver)
        spleen_label       = self.model3d_spleen(x_spleen)
        
        bowel_soft = self.softmax(bowel_label)
        extrav_soft = self.softmax(extrav_label)
        kidney_soft = self.softmax(kidney_label)
        liver_soft = self.softmax(liver_label)
        spleen_soft = self.softmax(spleen_label)
        
        labels = torch.cat([bowel_soft, extrav_soft, kidney_soft, liver_soft, spleen_soft], dim = 1)

        return labels

In [26]:
class Timm3DModelClassifierEmbed(nn.Module):
    def __init__(self, backbone, n_channels, n_labels, segtype='unet', pretrained=False):
        super(Timm3DModelClassifierEmbed, self).__init__()
        self.model_3d = Timm3DModel(backbone, n_channels, n_labels, segtype, pretrained)
        self.model_3d = convert_3d(self.model_3d)
        self.n_channels = n_channels
        self.n_labels = n_labels                        
        
    def forward(self, x):
        batch_size = x.shape[0]
        x = self.model_3d(x)
        pooled_features = []
        for i in range(0, len(x)):
            pooled_features.append(torch.reshape(torch.mean(x[i], dim = (2, 3, 4)), (batch_size, self.n_labels, 1)))
        pooled_features = torch.cat(pooled_features, dim=2)     
        labels = nn.Flatten()(pooled_features)
        #labels = torch.mean(pooled_features, dim = 2)
        return labels

# LSTM
class AbdominalClassifierLSTM(nn.Module):
    def __init__(self, device = DEVICE):
        super().__init__()
        self.device = device
        
        self.model3d_bowel        = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)      
        self.model3d_extrav       = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)
        self.model3d_kidney_left  = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)
        self.model3d_kidney_right = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)
        self.model3d_liver        = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)
        self.model3d_spleen       = Timm3DModelClassifierEmbed(backbone_classifiers3, 1, 32)
        
        self.flatten  = nn.Flatten()
        self.dropout  = nn.Dropout(p=0.5)
        self.softmax  = nn.Softmax(dim=1)        
        self.maxpool  = nn.MaxPool1d(5, 1)
        
        self.lstm = nn.LSTM(input_size =128, hidden_size = 256, num_layers=5, batch_first=True, bidirectional=True)
        self.head = nn.Linear(512, 13)
        
    def forward(self, x_bowel, x_kidney_left, x_kidney_right, x_liver, x_spleen, x_total):
        bs = x_bowel.shape[0]
        
        bowel_emb        = torch.reshape(self.model3d_bowel(x_bowel), (bs, 1, 128))
        extrav_emb       = torch.reshape(self.model3d_extrav(x_total), (bs, 1, 128))
        kidney_left_emb  = torch.reshape(self.model3d_kidney_left(x_kidney_left), (bs, 1, 128))
        kidney_right_emb = torch.reshape(self.model3d_kidney_right(x_kidney_right), (bs, 1, 128))
        liver_emb        = torch.reshape(self.model3d_liver(x_liver), (bs, 1, 128))
        spleen_emb       = torch.reshape(self.model3d_spleen(x_spleen), (bs, 1, 128))
        
        all_embs = torch.cat([bowel_emb, extrav_emb, kidney_left_emb, kidney_right_emb, liver_emb, spleen_emb], dim = 1)
        
        all_embs = self.lstm(all_embs)
        labels   = torch.mean(all_embs[0], dim = 1) 
        labels   = self.head(labels)

        bowel_soft = self.softmax(labels[:,:2])
        extrav_soft = self.softmax(labels[:,2:4])
        kidney_soft = self.softmax(labels[:,4:7])
        liver_soft = self.softmax(labels[:,7:10])
        spleen_soft = self.softmax(labels[:,10:13])

        labels = torch.cat([bowel_soft, extrav_soft, kidney_soft, liver_soft, spleen_soft], dim = 1)

        return labels

# Inference with cropped regions

In [27]:
class AbdominalCTDataset(Dataset):
    def __init__(self, meta_df, is_train = True, transform_set = None, remain_transforms_set = None):
        self.meta_df = meta_df
        self.is_train = is_train

    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, idx):
        row = self.meta_df.iloc[idx]        

        data_3ds = {}
        base_name = self.meta_df.iloc[idx]['cropped_path']            
        for j in range(0, 6):
            data_3d = decompress_fast(f'{base_name}_{j}').unsqueeze(0).to(torch.float32)
            #data_3d = torch.from_numpy(data_3d)
            data_3ds[chan_dict[j]] = data_3d  


        return data_3ds['bowel'], data_3ds['left_kidney'], data_3ds['right_kidney'], \
                data_3ds['liver'], data_3ds['spleen'], data_3ds['total']

test_dataset = AbdominalCTDataset(df_data)

In [28]:
# model = AbdominalClassifier()
# model.load_state_dict(torch.load(weights_classifier))
# model.to(DEVICE)    
# model.eval()
device = torch.device("cuda:0")  # for the first GPU
model = AbdominalClassifier() # resnet50
model.load_state_dict(torch.load(weights_classifier, map_location=device))
model.to(DEVICE)    
model.eval()

device = torch.device("cuda:0")  # for the first GPU
model1_2 = AbdominalClassifier() # resnet50
model1_2.load_state_dict(torch.load(weights_classifier1_2, map_location=device))
model1_2.to(DEVICE)    
model1_2.eval()


device = torch.device("cuda:0")  # for the first GPU
model2 = AbdominalClassifier2() # resnet10
model2.load_state_dict(torch.load(weights_classifier2, map_location=device))
model2.to(DEVICE)    
model2.eval()


device = torch.device("cuda:0")  # for the first GPU
model3 = AbdominalClassifierLSTM() # resnet10
model3.load_state_dict(torch.load(weights_classifiers3, map_location=device))
model3.to(DEVICE)    
model3.eval()

dummy=0

predss=[]
predss2 = []
predss1_2 = []
predss3 = []
#bar = tqdm(test_loader)

with torch.cuda.amp.autocast(enabled=False):  
    with torch.inference_mode():
        for i in tqdm(range(0, len(test_dataset))):
            X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot = test_dataset[i]
            X_bowel, X_lkid, X_rkid = X_bowel.unsqueeze(0).to(DEVICE), X_lkid.unsqueeze(0).to(DEVICE), X_rkid.unsqueeze(0).to(DEVICE)
            X_liv,   X_spl,  X_tot  = X_liv.unsqueeze(0).to(DEVICE),   X_spl.unsqueeze(0).to(DEVICE),  X_tot.unsqueeze(0).to(DEVICE)            
            preds = model(X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot).detach().cpu().numpy()  # model (resnet50)
            preds2 = model2(X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot).detach().cpu().numpy()  # model2 (resnet10)                        
            preds1_2 = model1_2(X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot).detach().cpu().numpy()  # model2 (resnet10)
            preds3 = model3(X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot).detach().cpu().numpy()  # LSTM

            predss.append(preds)
            predss2.append(preds2)
            predss1_2.append(preds1_2)
            predss3.append(preds3)
            del X_bowel, X_lkid, X_rkid, X_liv, X_spl, X_tot
            gc.collect()
            
        

predss = np.vstack(predss)
predss2 = np.vstack(predss2)
predss1_2 = np.vstack(predss1_2)
predss3 = np.vstack(predss3)
                
#del images
#gc.collect()
torch.cuda.empty_cache()

target_cols =  ['bowel_healthy','bowel_injury',
                    'extravasation_healthy','extravasation_injury',
                    'kidney_healthy','kidney_low','kidney_high',
                    'liver_healthy','liver_low','liver_high',
                    'spleen_healthy','spleen_low','spleen_high']

# avg preds
avg_preds = (predss + predss2 + predss1_2+predss3) / 4

df_data[target_cols] = avg_preds
#preds = np.vstack(preds)            

try:
    df_data = df_data.drop('series', axis = 1)
except:
    df_data = df_data.drop('sample', axis = 1)

# More sofisticated postprocessing    
bowel_extrav =  \
            df_data.groupby(['patient_id']).agg('max')[['bowel_injury', 'extravasation_injury']].to_numpy()

df_data = df_data.groupby(['patient_id']).agg('mean')

df_data['bowel_healthy'] = 1 - bowel_extrav[:,0]
df_data['bowel_injury']  = bowel_extrav[:,0]
df_data['extravasation_healthy'] = 1- bowel_extrav[:,1]
df_data['extravasation_injury'] =  bowel_extrav[:,1]

patient_id = df_data.index
df_data = df_data.reset_index()

df_data['patient_id'] = patient_id

df_data

100%|██████████| 3/3 [00:02<00:00,  1.01it/s]
/tmp/ipykernel_22/2150712000.py:89: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_data = df_data.groupby(['patient_id']).agg('mean')


,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.892306,0.107694,0.511987,0.488013,0.560934,0.296946,0.142119,0.943546,0.035143,0.021312,0.911876,0.068158,0.019965
1,50046,0.866436,0.133564,0.481671,0.518329,0.615959,0.248909,0.135132,0.755278,0.068269,0.176453,0.832170,0.140618,0.027212
2,63706,0.948301,0.051699,0.631876,0.368124,0.601788,0.274645,0.123567,0.886764,0.081744,0.031492,0.754351,0.212043,0.033606


In [29]:
sample_df = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv')
if not DEBUG:
    for i in range(0, len(df_data)):
        row = df_data.iloc[i]
        patient_id = row['patient_id']
        sample_df.loc[sample_df['patient_id']==patient_id,target_cols] = row[target_cols].to_numpy()
#Extravasation 
sample_df.to_csv('submission.csv', index = False)
sample_df

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high
0,48843,0.892306,0.107694,0.511987,0.488013,0.560934,0.296946,0.142119,0.943546,0.035143,0.021312,0.911876,0.068158,0.019965
1,50046,0.866436,0.133564,0.481671,0.518329,0.615959,0.248909,0.135132,0.755278,0.068269,0.176453,0.832170,0.140618,0.027212
2,63706,0.948301,0.051699,0.631876,0.368124,0.601788,0.274645,0.123567,0.886764,0.081744,0.031492,0.754351,0.212043,0.033606
